In [1]:
lan = ["eng", "esp", "amh","arq","ary","hau","kin","mar","tel"]
# lan="eng"
#sentence-transformers/distiluse-base-multilingual-cased-v2

In [2]:

num_labels=1
seed=7
batch_size=16
lr=2e-5
# set the number of epochs
n_epochs = 5 #3
l_lr=5e-2
emded_d = 1024
max_len=92
early_stoping_patience=3


In [3]:
import random

random.seed(seed)


In [6]:
import re
import tqdm
tqdmn = tqdm.notebook.tqdm
import zipfile
import pandas as pd
# from tqdm import tqdmn
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [7]:
from transformers import  XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaForSequenceClassification


In [5]:

# -*- coding: utf-8 -*-


"""# Import the Libraries"""
import pandas as pd
from collections import Counter
import sys
import json
import json
# visualization libraries
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# pytorch libraries
import torch # the main pytorch library
import torch.nn as nn # the sub-library containing Softmax, Module and other useful functions
import torch.optim as optim # the sub-library containing the common optimizers (SGD, Adam, etc.)

# huggingface's transformers library
from transformers import RobertaForSequenceClassification, AutoTokenizer

# huggingface's datasets library
from datasets import load_dataset
from datasets import  load_from_disk

# the tqdm library used to show the iteration progress
import tqdm
from tqdm import tqdm
# tqdmn = tqdm.notebook.tqdm#tqdm.tqdm# tqdm.notebook.tqdm

from datasets import load_dataset
import os
from transformers import get_scheduler
import torch
from datasets import Dataset
from datasets import load_from_disk
import sys
from sklearn.metrics import confusion_matrix
import copy

In [9]:
# train Load the File
# df_final=pd.DataFrame()
# df_final['PairID']=None
# df_final['Text']=None

for i, lang in enumerate(lan):
    df_str_rel_test = pd.read_csv("Track A/"+lang+"/"+lang+"_test.csv")
    df_final=pd.concat([df_str_rel_test, df_final])
 

FileNotFoundError: [Errno 2] No such file or directory: 'Track A/eng/eng_test.csv'

In [12]:
df_str_rel_train.keys()

dict_keys(['eng', 'esp', 'amh', 'arq', 'ary', 'hau', 'kin', 'mar', 'tel'])

In [13]:
# train Load the File
df_str_rel_dev={}
# df_final=pd.DataFrame()
for i, lang in enumerate(lan):
    df_str_rel_dev[lang]= pd.read_csv("Track A/"+lang+"/"+lang+"_dev.csv")

In [14]:
df_str_rel_dev['eng']

,PairID,Text
0,ENG-dev-0000,The story is gripping and interesting.\nIt's a...
1,ENG-dev-0001,The majority of Southeast Alaska 's area is pa...
2,ENG-dev-0002,and from your post i think you are to young to...
3,ENG-dev-0003,The film 's success also made Dreamworks Anima...
4,ENG-dev-0004,I am still confused about how I feel about thi...
...,...,...
245,ENG-dev-0245,thats just how they are :( its a shame lol\nIt...
246,ENG-dev-0246,I feel sorry for the books that I will read af...
247,ENG-dev-0247,Uwe Seeler -LRB- born 5 November 1936 in Hambu...
248,ENG-dev-0248,Waco is a city in and the county seat of McLen...


In [15]:
# # Creating a column "Split_Text" which is a list of two sentences.

# df_final['Split_Text'] = df_final['Text'].apply(lambda x: x.split("\n"))
# df_final.head()
for i, lang in enumerate(lan):
    df_str_rel_train[lang]['Split_Text']= df_str_rel_train[lang]['Text'].apply(lambda x:x.split("\n"))
    df_str_rel_dev[lang]['Split_Text']= df_str_rel_dev[lang]['Text'].apply(lambda x:x.split("\n"))

In [16]:
df_str_rel_train['eng']

,PairID,Text,Score,Split_Text
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"[It that happens, just pull the plug., if that..."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,"[A black dog running through water., A black d..."
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,"[I've been searchingthe entire abbey for you.,..."
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,[If he is good looking and has a good personal...
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"[She does not hate you, she is just annoyed wi..."
...,...,...,...,...
5495,ENG-train-5495,A young boy pounding on an anvil.\nWoman sits ...,0.0,"[A young boy pounding on an anvil., Woman sits..."
5496,ENG-train-5496,I love how he recognized his wife tempered his...,0.0,[I love how he recognized his wife tempered hi...
5497,ENG-train-5497,I actually read a chapter or two beyond that p...,0.0,[I actually read a chapter or two beyond that ...
5498,ENG-train-5498,A boy gives being in the snow two thumbs up.\n...,0.0,"[A boy gives being in the snow two thumbs up.,..."


#Generate submission file
```
# This is formatted as code
```



### Append prediction to dataframe

> Indented block

In [17]:
# train_pdf=df_final.sample(frac=0.9,random_state=200)
# dev_pdf=df_final.drop(train_pdf.index)

test_pdf = df_str_rel_dev
train_pdf={}
dev_pdf={}
for i, lang in enumerate(lan):
    train_pdf[lang]= df_str_rel_train[lang].sample(frac=0.9, random_state=200)
    dev_pdf[lang]=df_str_rel_train[lang].drop(train_pdf[lang].index)


In [18]:
train_pdf['eng']

,PairID,Text,Score,Split_Text
5411,ENG-train-5411,Stowers worked as a receptionist in Los Angele...,0.09,[Stowers worked as a receptionist in Los Angel...
2224,ENG-train-2224,"Use a search engine like google or yahoo, and ...",0.53,"[Use a search engine like google or yahoo, and..."
2294,ENG-train-2294,Must add Gwen and Drustan to my list of all-ti...,0.53,[Must add Gwen and Drustan to my list of all-t...
5229,ENG-train-5229,How am I going to last till June!\nUPDATE: thi...,0.16,"[How am I going to last till June!, UPDATE: th..."
139,ENG-train-0139,"We're already two million, two over the origin...",0.91,"[We're already two million, two over the origi..."
...,...,...,...,...
4919,ENG-train-4919,My mom used to listen to Karen Carpenter often...,0.22,[My mom used to listen to Karen Carpenter ofte...
5296,ENG-train-5296,He left an apologetic message.\nIt's below the...,0.12,"[He left an apologetic message., It's below th..."
3714,ENG-train-3714,I don't know Cynna well enough for this.\nCynn...,0.38,"[I don't know Cynna well enough for this., Cyn..."
3718,ENG-train-3718,"Hades and Persephone are each likable, decent ...",0.38,"[Hades and Persephone are each likable, decent..."


In [19]:
test_pdf['eng']

,PairID,Text,Split_Text
0,ENG-dev-0000,The story is gripping and interesting.\nIt's a...,"[The story is gripping and interesting., It's ..."
1,ENG-dev-0001,The majority of Southeast Alaska 's area is pa...,[The majority of Southeast Alaska 's area is p...
2,ENG-dev-0002,and from your post i think you are to young to...,[and from your post i think you are to young t...
3,ENG-dev-0003,The film 's success also made Dreamworks Anima...,[The film 's success also made Dreamworks Anim...
4,ENG-dev-0004,I am still confused about how I feel about thi...,[I am still confused about how I feel about th...
...,...,...,...
245,ENG-dev-0245,thats just how they are :( its a shame lol\nIt...,"[thats just how they are :( its a shame lol, I..."
246,ENG-dev-0246,I feel sorry for the books that I will read af...,[I feel sorry for the books that I will read a...
247,ENG-dev-0247,Uwe Seeler -LRB- born 5 November 1936 in Hambu...,[Uwe Seeler -LRB- born 5 November 1936 in Hamb...
248,ENG-dev-0248,Waco is a city in and the county seat of McLen...,[Waco is a city in and the county seat of McLe...


In [20]:
dev_pdf

{'eng':               PairID                                               Text  \
 17    ENG-train-0017  Fritz Laband was a German footballer .\nFritz ...   
 67    ENG-train-0067  Their address is: 47483 Conneticut Dr, Nashvil...   
 76    ENG-train-0076  The sequel , Pikmin 2 , was released in 2004 ....   
 106   ENG-train-0106  Game Maker is a Windows computer program origi...   
 111   ENG-train-0111  Take him aside and tell him how you truly feel...   
 ...              ...                                                ...   
 5424  ENG-train-5424  Anything to the east?" the CO persisted.\nKing...   
 5430  ENG-train-5430  These noodles on my head are so hard to comb!\...   
 5443  ENG-train-5443  Before he gets out of control.\nand she gets p...   
 5473  ENG-train-5473  Eight heads in a duffle bag.\nFor someone my a...   
 5478  ENG-train-5478  It involves balls of some type?\nSelect "run" ...   
 
       Score                                         Split_Text  
 17     1.00 

In [21]:
# valid_true_labels= dev_pdf["Score"]
valid_true_labels={}
for i, lang in enumerate(lan):
    valid_true_labels[lang]= dev_pdf[lang]['Score']

In [22]:
valid_true_labels

{'eng': 17      1.00
 67      0.94
 76      0.94
 106     0.94
 111     0.94
         ... 
 5424    0.09
 5430    0.07
 5443    0.06
 5473    0.06
 5478    0.03
 Name: Score, Length: 550, dtype: float64,
 'esp': 0       1.00
 29      0.47
 52      0.37
 67      0.15
 76      0.61
         ... 
 1522    0.25
 1529    0.71
 1536    0.76
 1537    0.73
 1553    0.22
 Name: Score, Length: 156, dtype: float64,
 'amh': 0      0.88
 6      0.27
 7      0.20
 42     0.95
 51     0.73
        ... 
 962    0.22
 980    0.08
 983    0.12
 988    0.27
 989    0.59
 Name: Score, Length: 99, dtype: float64,
 'arq': 0       0.62
 17      0.34
 29      0.50
 52      0.69
 67      0.31
         ... 
 1215    0.50
 1223    0.47
 1225    0.31
 1229    0.56
 1252    0.69
 Name: Score, Length: 126, dtype: float64,
 'ary': 0      0.38
 6      0.39
 7      0.66
 42     0.18
 51     0.56
        ... 
 902    0.25
 912    0.28
 915    0.71
 920    0.57
 921    0.75
 Name: Score, Length: 92, dtype: float64,
 'ha

In [23]:
test_pdf

{'eng':            PairID                                               Text  \
 0    ENG-dev-0000  The story is gripping and interesting.\nIt's a...   
 1    ENG-dev-0001  The majority of Southeast Alaska 's area is pa...   
 2    ENG-dev-0002  and from your post i think you are to young to...   
 3    ENG-dev-0003  The film 's success also made Dreamworks Anima...   
 4    ENG-dev-0004  I am still confused about how I feel about thi...   
 ..            ...                                                ...   
 245  ENG-dev-0245  thats just how they are :( its a shame lol\nIt...   
 246  ENG-dev-0246  I feel sorry for the books that I will read af...   
 247  ENG-dev-0247  Uwe Seeler -LRB- born 5 November 1936 in Hambu...   
 248  ENG-dev-0248  Waco is a city in and the county seat of McLen...   
 249  ENG-dev-0249  religious extremism continues in pakistan desp...   
 
                                             Split_Text  
 0    [The story is gripping and interesting., It's ...  

In [24]:
train_dataset={}
valid_dataset={}
for i, lang in enumerate(lan):
    train_dataset[lang] =  Dataset.from_pandas(train_pdf[lang])
    train_dataset[lang] =  train_dataset[lang].rename_column("PairID", "sentence_id")
    train_dataset[lang] =  train_dataset[lang].rename_column("Text", "words")
    
    
    valid_dataset[lang] = Dataset.from_pandas(dev_pdf[lang])
    valid_dataset[lang] = valid_dataset[lang].rename_column("PairID", "sentence_id")
    valid_dataset[lang] = valid_dataset[lang].rename_column("Text", "words")



In [25]:
test_dataset={}
for lang in lan:
    test_dataset[lang] =  Dataset.from_pandas(test_pdf[lang])
    test_dataset[lang] = test_dataset[lang].rename_column("PairID", "sentence_id")
    test_dataset[lang] = test_dataset[lang].rename_column("Text", "words")
    test_dataset

In [26]:
test_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 293
 }),
 'tel': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 130
 })}

In [27]:
train_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 4950
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 1406
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 893
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 1135
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 832
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 1562
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 700
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
    

In [28]:
def sentence_len_distribution(sentences):
    sentences_len=[]

    for sentence in sentences:
        sentences_len.append(len(sentence))
    return Counter(sentences_len)

In [29]:
test_dataset['eng']["words"][0]

"The story is gripping and interesting.\nIt's a brilliant, compelling, and heartfelt story."

In [30]:
train_dataset['amh']["Split_Text"]

[['” የሚለው ነው ዋናው ጥያቄ ።\tያለው የሚለው ጥያቄ ለህዝቡ መመለስ ግዴታ ነው ።'],
 ['አቶ ነጋ ፡ (ረዥም ሳቅ) እንዴት ነው እንዴ ?\tአቶ ነጋ ፡ (ሳቅ) በቃ ተወው ።'],
 ['መልካም አዲስ አመት ቀድሜ ፣ተሽቀዳድሜ ።\tመልካም አዲስ አመት ይሁንልን ::'],
 ['ልጄ ልብ ያለው ልብ ቢል አሪፍ ነው ፡፡\tእንደ አንተ ልብ ሁሉ የእኔም ልብ እንዲሁ ነው አለው ።'],
 ['እርስ-በእርስ መጠላለፍና መዘላለፍ ከጀመርን ግን አሁን ያለውን የለውጥ ጭላጭል ያዳፍነዋል ።\tበማንኩሳ ዝርዝር ጉዳዮችን በስልክ መቆራረጥ ምክንያት ማግኘት አልቻልንም ፡፡'],
 ['የኢትዮጵያ ሙዚየም ማውጫ ታትሞ ወጣ ።\t“የኢትዮጵያ ህያው ሙዚየም” የተሰኘ የሙዚየም ማውጫ ታትሞ ወጣ ፡፡'],
 ['የኖቤል የሰላም ሽልማትን ካገኙ ሰዎች መካከል ማንዴላ አንዱ ናቸው ።\tጠቅላይ ሚኒስተር ዐብይ አሕመድ የኖቤል የሰላም ሽልማትን በተቀበሉበት ጊዜ ።'],
 ['የምድር ባቡር ግንባታ ሥራ ተቆጣጣሪ[ ለማስተካከል | ኮድ አርም ] ።\tመተላለፊያው[ ለማስተካከል | ኮድ አርም ] ።'],
 ['በጣም አስፈላጊ ምርጫ ነዉ ።\tእንደ 2016ቱ በጣም አስፈላጊ ነዉ አልልም ።'],
 ['" የጠንቋዮችና ፡ የደጋሚዎች እናም የገጣሚዎችን ንግግር ሰምቻለሁ ።\tለነገሩ እኔም ሰው ነግሮኝ ነው ነው ?'],
 ['ኢትዮጵያ እና ኤርትራ ከማን ወገን ናቸው ?\tኢትዮጵያ እና ኤርትራ ምን ሚና ሊኖራቸው ይችላል ?'],
 ['ዳና ድራማ ክፍል 61 ፣ ይመልከቱ ።\tቤቶች ድራማ ክፍል 179 ፣ ይመልከቱ ።'],
 ['ወታደራዊው መንግስት ስልጣን ከመያዙ ሁለት ዓመት በፊት ነው የተወለድኩት ትላለች ታሪክ እየጠቀሰች ።\tየኢሕአዴግ አባል ፓርቲዎች እርስበርሳቸው እየተዋቀሱ ‘አጋር’ ከሚሏቸው ፓርቲዎች ጋር ተስማምተ

In [31]:
# !pip install -U sentence-transformers

# from sentence_transformers import SentenceTransformer, util

In [32]:
# model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


In [33]:

# sentences = ["I'm happy", "I'm full of happiness"]


# #Compute embedding for both lists
# embedding_1= model.encode(sentences[0], convert_to_tensor=True)
# embedding_2 = model.encode(sentences[1], convert_to_tensor=True)

# util.pytorch_cos_sim(embedding_1, embedding_2)


Train data


In [34]:
# true_scores = train_pdf['Score'].values
# pred_scores = []

# for index,row in train_pdf.iterrows():
#   #Compute embedding for both lists
#   embedding_1= model.encode(row["Split_Text"][0], convert_to_tensor=True)
#   embedding_2 = model.encode(row["Split_Text"][1], convert_to_tensor=True)


#   # Overlap score
#   pred_scores.append(util.pytorch_cos_sim(embedding_1, embedding_2))

In [35]:
# pred=[]
# for i in range(len(pred_scores)):
#   pred.append(float(pred_scores[i][0]))
# pred_scores=pred

In [36]:
# for i in range(len(pred_scores)):
#   pred_scores[i] =float(pred_scores[i])

test data

In [37]:
train_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 4950
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 1406
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 893
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 1135
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 832
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 1562
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
     num_rows: 700
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
    

In [38]:
train_pdf['Split_Text'][1]

KeyError: 'Split_Text'

In [39]:
# test_pdf[['PairID', 'Pred_Score']].to_csv('pred_eng.csv', index=False)

In [40]:
roberta_version = 'xlm-roberta-large'#"/home/naive123/nlp/Sumit/models_repo/muril_large_model"#
roberta_version_tokenizer = 'xlm-roberta-large'#"/home/naive123/nlp/Sumit/models_repo/muril_large_tokenizer"
tokenizer = AutoTokenizer.from_pretrained(roberta_version_tokenizer)
tokenizer.model_max_length = max_len
print(tokenizer.model_max_length )

92


In [41]:
train_dataset['eng']

Dataset({
    features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
    num_rows: 4950
})

In [42]:
test_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 293
 }),
 'tel': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 130
 })}

In [43]:
# #Cuda device


# not_equal=0
# word_ids_of_sentences = []
# repeat_batch=0


# valid_tokenization = []
# npt_equql_flag = 0
# def add_encodings(example):if not os.path.exists(zip_directory):
        # os.makedirs(zip_directory)

#     global npt_equql_flag
#     global word_ids_of_sentences
#     global repeat_batch
#     example_words={}
#     example={}
#     encodings={}
#     score={}
#     for i, lang in enumerate(lan):

#         example_words=  example[lang]['words']   # for bert type models
    
#         # atoc=tokenizer.convert_ids_to_tokens(tokenizer(example['words'],is_split_into_words=True)["input_ids"],skip_special_tokens=False)[1:-1]
#         # print(example)
#         # if len(example['Split_Text'])!=2:
#         #     example['Split_Text']=["not equal", "not equal"]
#         #     not_equal+=1
#         # print(example)
#         print(example[lang]["sentence_id"])
#         # try:
#         #     if example["sentence_id"][8:11]=='amh':
#         #         example['Split_Text']=example['Split_Text'].split("\t")
#         #         print(example['Split_Text'])
#         #         encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
#         #     else:
#         #         encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
#         #     #print(encodings)
#         # except:
#         #     print('error')
#         #     print(example)
#         if example[lang]["sentence_id"][8:11]=='amh':
#             print(example[lang]['Split_Text'])
#             example['Split_Text']=example['Split_Text'][0].split("\t")
#             print(example[lang]['Split_Text'])
#             encodings[lang] = tokenizer(example[lang]['Split_Text'][0]+"</s>"+example[lang]['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
#         else:
#             encodings[lang] = tokenizer(example[lang]['Split_Text'][0]+"</s>"+example[lang]['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
#         #print(encodings)
#         if "Score" in example[lang].keys():
#           score[lang]=example[lang]['Score']
#         else:
#           score=0
    
#         # return the encodings and the extended ner_tags
#         return { **encodings[lang], 'labels': score[lang]}


In [44]:
#Cuda device


not_equal=0
word_ids_of_sentences = []
repeat_batch=0


valid_tokenization = []
npt_equql_flag = 0
def add_encodings(example):
    global npt_equql_flag
    global word_ids_of_sentences
    global repeat_batch


    example_words=  example['words']   # for bert type models

    # atoc=tokenizer.convert_ids_to_tokens(tokenizer(example['words'],is_split_into_words=True)["input_ids"],skip_special_tokens=False)[1:-1]
    # print(example)
    # if len(example['Split_Text'])!=2:
    #     example['Split_Text']=["not equal", "not equal"]
    #     not_equal+=1
    # print(example)
    # print(example["sentence_id"])
    # try:
    #     if example["sentence_id"][8:11]=='amh':
    #         example['Split_Text']=example['Split_Text'].split("\t")
    #         print(example['Split_Text'])
    #         encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    #     else:
    #         encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    #     #print(encodings)
    # except:
    #     print('error')
    #     print(example)
    if example["sentence_id"][8:11]=='amh':
        # print(example['Split_Text'])
        example['Split_Text']=example['Split_Text'][0].split("\t")
        # print(example['Split_Text'])
        encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    else:
        encodings = tokenizer(example['Split_Text'][0]+"</s>"+example['Split_Text'][1], truncation=True, padding='max_length', is_split_into_words=False)
    #print(encodings)
    if "Score" in example.keys():
      score=example['Score']
    else:
      score=0

    # return the encodings and the extended ner_tags
    return { **encodings, 'labels': score}


In [45]:
train_dataset
train_dataset["eng"]

Dataset({
    features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__'],
    num_rows: 4950
})

In [46]:
# train_dataset["eng"].map(add_encodings,load_from_cache_file=False, batch_size =10000)

In [47]:
print("tokenizer.model_max_length=",tokenizer.model_max_length)

print("encodinmmg start")
#valid_dataset = valid_dataset.map(add_encodings)
# print("encodinmmg complete")
#aaa=add_encodings(train_dataset[1166])
#aaa=add_encodings(train_dataset[1167])
train_data={}
print("encodinmmg start")
for i, lang in enumerate(lan):
    # print(lang)
    # print(train_dataset[lang].num_rows)
    train_data[lang] = train_dataset[lang].map(add_encodings,load_from_cache_file=False, batch_size =10000)#load_from_cache_file=False,
    #train_subwordid_1s, train_subwordid_2s, train_subwordid_3s = create_Subwordid(word_ids_for_curr_sentence)
    print(train_data[lang][0])
    # print(train_data[lang][1])
    # print(train_data[lang][2])
    print("encodinmmg complete")

tokenizer.model_max_length= 92
encodinmmg start
encodinmmg start


Map:   0%|          | 0/4950 [00:00<?, ? examples/s]

{'sentence_id': 'ENG-train-5411', 'words': 'Stowers worked as a receptionist in Los Angeles .\nAnimators also visited the Los Angeles Zoo .', 'Score': 0.09, 'Split_Text': ['Stowers worked as a receptionist in Los Angeles .', 'Animators also visited the Los Angeles Zoo .'], '__index_level_0__': 5411, 'input_ids': [0, 18343, 42844, 79786, 237, 10, 116816, 1419, 23, 3731, 31754, 6, 5, 2, 71352, 22230, 2843, 36997, 71, 70, 3731, 31754, 70755, 6, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0.09}
encodinmmg complete


Map:   0%|          | 0/1406 [00:00<?, ? examples/s]

{'sentence_id': 'ESP-train-1485', 'words': 'Un hombre árabe lleva un sombrero blanco\nLa niña anima al hombre del uniforme azul y blanco.', 'Score': 0.5, 'Split_Text': ['Un hombre árabe lleva un sombrero blanco', 'La niña anima al hombre del uniforme azul y blanco.'], '__index_level_0__': 1485, 'input_ids': [0, 992, 46491, 193390, 28606, 51, 124, 4134, 516, 101455, 2, 239, 300, 5708, 10127, 144, 46491, 146, 125739, 80218, 113, 101455, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0.5}
encodinmmg complete


...

{'sentence_id': 'kin_train_00617', 'words': 'Byari agahebuzo ubwo Ruhago Sports Promoters na Rayon Sports batangizaga ku ku mugaragaro ubufatanye bwabo!\nTubibutse ko Ruhago Promoters LTD yatangiye ubufatanye n’ikipe ya Rayon Sports buzamara imyaka 5', 'Score': 0.78, 'Split_Text': ['Byari agahebuzo ubwo Ruhago Sports Promoters na Rayon Sports batangizaga ku ku mugaragaro ubufatanye bwabo!', 'Tubibutse ko Ruhago Promoters LTD yatangiye ubufatanye n’ikipe ya Rayon Sports buzamara imyaka 5'], '__index_level_0__': 616, 'input_ids': [0, 3311, 1686, 5575, 1106, 978, 1158, 6, 5829, 3613, 82975, 13906, 39170, 23490, 23962, 24, 73706, 39170, 38880, 7337, 208, 228, 228, 83360, 42547, 516, 74166, 136208, 13838, 90998, 21552, 38, 2, 1371, 964, 8789, 184, 298, 82975, 13906, 23490, 23962, 111405, 151, 14525, 8380, 74166, 136208, 13838, 653, 26, 5898, 1081, 151, 73706, 39170, 57461, 109348, 566, 122669, 190, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],

Map:   0%|          | 0/1080 [00:00<?, ? examples/s]

{'sentence_id': 'MAR-train-00547', 'words': '"त्यामुळे आमची युती झालीय हे घरोघरी सांगण्याची जबाबदारी उद्धव यांनी आज जिल्हा आणि तालुका पातळीवरील आपल्या शिलेदारांवर सोपवली आहे."\n"मात्र, रात्री उशिरा वा उद्या मुख्यमंत्री देवेंद्र फडणवीस हे शिवसेना पक्षप्रमुख उद्धव ठाकरे यांच्याशी चर्चा करून तोडगा काढण्याची शक्यता आहे."', 'Score': 0.5, 'Split_Text': ['"त्यामुळे आमची युती झालीय हे घरोघरी सांगण्याची जबाबदारी उद्धव यांनी आज जिल्हा आणि तालुका पातळीवरील आपल्या शिलेदारांवर सोपवली आहे."', '"मात्र, रात्री उशिरा वा उद्या मुख्यमंत्री देवेंद्र फडणवीस हे शिवसेना पक्षप्रमुख उद्धव ठाकरे यांच्याशी चर्चा करून तोडगा काढण्याची शक्यता आहे."'], '__index_level_0__': 546, 'input_ids': [0, 44, 23289, 47163, 26046, 3673, 32479, 3801, 30831, 3346, 3653, 5988, 2284, 68578, 659, 26844, 36328, 181353, 659, 226291, 15325, 3264, 55211, 2056, 242562, 6, 67377, 24718, 80933, 17375, 26502, 535, 11639, 84749, 11110, 2138, 95826, 1383, 1242, 2, 44, 64407, 4, 126501, 5450, 31771, 2815, 5093, 160242, 32894, 195323, 190569, 3

Map:   0%|          | 0/1053 [00:00<?, ? examples/s]

{'sentence_id': 'TEL-train-00681', 'words': '"ఈ చిత్రం ఫై అటు అక్కినేని అభిమానులకే కాక ఇటు సగటు సినిమా ప్రేక్షకులకు కూడా భారీ అంచనాలు ఉన్నాయి."\n"చదివింది భీమవరం అయితే, ఇప్పుడు బ్రతుకుతున్నది మాత్రం పూరి గారి మీద పడి అని, తను పూరి గారి భక్తుడినని ఎందుకంటే ఒక గురువుకి శిష్యుడిని మించిన భక్తుడు ఎవ్వరూ ఉండరని, అలాగే ఒక శిష్యుడికి గురువుకి మించిన గొప్పది ఏదీ ఈ ప్రపంచంలో లేదని అన్నారు."', 'Score': 0.44, 'Split_Text': ['"ఈ చిత్రం ఫై అటు అక్కినేని అభిమానులకే కాక ఇటు సగటు సినిమా ప్రేక్షకులకు కూడా భారీ అంచనాలు ఉన్నాయి."', '"చదివింది భీమవరం అయితే, ఇప్పుడు బ్రతుకుతున్నది మాత్రం పూరి గారి మీద పడి అని, తను పూరి గారి భక్తుడినని ఎందుకంటే ఒక గురువుకి శిష్యుడిని మించిన భక్తుడు ఎవ్వరూ ఉండరని, అలాగే ఒక శిష్యుడికి గురువుకి మించిన గొప్పది ఏదీ ఈ ప్రపంచంలో లేదని అన్నారు."'], '__index_level_0__': 680, 'input_ids': [0, 44, 56223, 30784, 110962, 155578, 136992, 170884, 182811, 15476, 140173, 7377, 16685, 8653, 4701, 16685, 6353, 131238, 2106, 5253, 49587, 145252, 1361, 53107, 1242, 2, 44, 14221, 6204, 2502, 938

In [48]:
train_data

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 4950
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1406
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 893
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 1135
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 832
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Score', 'Split_Text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
     num_rows

In [49]:
test_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 293
 }),
 'tel': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 130
 })}

In [50]:
lan

['eng', 'esp', 'amh', 'arq', 'ary', 'hau', 'kin', 'mar', 'tel']

In [51]:
test_data={}
for i, lang in enumerate(lan):
    # print(lang)
    # print(train_dataset[lang].num_rows)
    test_data[lang] = test_dataset[lang].map(add_encodings,load_from_cache_file=False, batch_size =10000)#load_from_cache_file=False,
    #train_subwordid_1s, train_subwordid_2s, train_subwordid_3s = create_Subwordid(word_ids_for_curr_sentence)
    print(test_data[lang][0])
    # print(train_data[lang][1])
    # print(train_data[lang][2])
    print("encodinmmg complete")
for i, lang in enumerate(lan):
    test_data[lang].set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

{'sentence_id': 'ENG-dev-0000', 'words': "The story is gripping and interesting.\nIt's a brilliant, compelling, and heartfelt story.", 'Split_Text': ['The story is gripping and interesting.', "It's a brilliant, compelling, and heartfelt story."], 'input_ids': [0, 581, 13765, 83, 10314, 26783, 136, 49041, 5, 2, 1650, 25, 7, 10, 230451, 4, 9969, 13, 30319, 4, 136, 26498, 117905, 13765, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}
encodinmmg complete


Map:   0%|          | 0/140 [00:00<?, ? examples/s]

{'sentence_id': 'ESP-dev-0000', 'words': 'Notable es la carta de Robert a Sarah.\nStrathairn asistió a Williams College, Williamstown, Massachusetts, y se graduó de la Redwood High School en Larkspur, California en 1970.', 'Split_Text': ['Notable es la carta de Robert a Sarah.', 'Strathairn asistió a Williams College, Williamstown, Massachusetts, y se graduó de la Redwood High School en Larkspur, California en 1970.'], 'input_ids': [0, 438, 22819, 198, 21, 20533, 8, 12452, 10, 44582, 5, 2, 7832, 48625, 19, 96250, 12632, 10, 40478, 29693, 4, 40478, 35461, 4, 157082, 4, 113, 40, 25975, 849, 8, 21, 6096, 25876, 19694, 19188, 22, 239, 7190, 48150, 4, 39897, 22, 19340, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

Map:   0%|          | 0/95 [00:00<?, ? examples/s]

...

{'sentence_id': 'kin_dev_00001', 'words': "Impamvu zo kwandura indwara zo mu gitsina : Ukumagara mu gitsina kandi bishobora guterwa n'indwara zifata mu gitsina.\nImpamvu z'ibiyobyabwenge : Itabi, alukolo nyinshi (alcool) bishobora na byo gutera ukumagara mu gitsina biherekejwe no kunanirwa k'ubwonko, no kugira umunaniro umubiri wose.", 'Split_Text': ["Impamvu zo kwandura indwara zo mu gitsina : Ukumagara mu gitsina kandi bishobora guterwa n'indwara zifata mu gitsina.", "Impamvu z'ibiyobyabwenge : Itabi, alukolo nyinshi (alcool) bishobora na byo gutera ukumagara mu gitsina biherekejwe no kunanirwa k'ubwonko, no kugira umunaniro umubiri wose."], 'input_ids': [0, 29128, 302, 1783, 1602, 760, 46768, 11, 2752, 63991, 1602, 842, 62270, 30529, 152, 36795, 192, 17117, 842, 62270, 30529, 203, 428, 2464, 497, 27651, 129278, 634, 653, 25, 7250, 63991, 3546, 136208, 842, 62270, 30529, 5, 2, 29128, 302, 1783, 97, 25, 14, 40165, 45104, 2055, 11697, 429, 152, 1650, 14508, 4, 64344, 45898, 600, 73, 37

Map:   0%|          | 0/293 [00:00<?, ? examples/s]

{'sentence_id': 'MAR-dev-00001', 'words': '"या प्रसंगी जाहीर सभेला संबोधित करताना ते म्हणाले की, नऊ उपक्रम आहेत, ज्यात 500 कोटी रुपयांची किंमत आहे, जे आज एकतर समर्पित आहेत, किंवा त्यांचे पाया दगड घातले जात आहेत."\n"आज 500 कोटी रुपयांच्या नऊ उपक्रमांचे आज लोकार्पण अथवा भूमीपुजन करण्यात आले असे पंतप्रधानांनी यावेळी जाहीर सभेत बोलताना सांगितले."', 'Split_Text': ['"या प्रसंगी जाहीर सभेला संबोधित करताना ते म्हणाले की, नऊ उपक्रम आहेत, ज्यात 500 कोटी रुपयांची किंमत आहे, जे आज एकतर समर्पित आहेत, किंवा त्यांचे पाया दगड घातले जात आहेत."', '"आज 500 कोटी रुपयांच्या नऊ उपक्रमांचे आज लोकार्पण अथवा भूमीपुजन करण्यात आले असे पंतप्रधानांनी यावेळी जाहीर सभेत बोलताना सांगितले."'], 'input_ids': [0, 44, 3889, 186353, 659, 150442, 2218, 65758, 1404, 241005, 128611, 4429, 111738, 471, 4, 1618, 65164, 68097, 7419, 4, 43135, 996, 2101, 90676, 133038, 3673, 135489, 1383, 4, 16478, 3264, 967, 21100, 218792, 7419, 4, 30435, 115593, 111484, 226415, 6, 64239, 535, 35951, 7419, 1242, 2, 44, 153649, 2101, 90676, 13303

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

{'sentence_id': 'TEL-dev-00001', 'words': '"బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా నేడు రెండోరోజు నల్గొండలో పర్యటించనున్నారు ."\n"బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా గారు ఈ రోజు రెండవసారి నల్గొండని సందర్శించనున్నారు."', 'Split_Text': ['"బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా నేడు రెండోరోజు నల్గొండలో పర్యటించనున్నారు ."', '"బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా గారు ఈ రోజు రెండవసారి నల్గొండని సందర్శించనున్నారు."'], 'input_ids': [0, 44, 59380, 48677, 37774, 68659, 157121, 3409, 10428, 10511, 59924, 160220, 116655, 78352, 6, 101486, 231824, 1296, 72738, 36111, 7747, 14710, 156045, 6, 1242, 2, 44, 59380, 48677, 37774, 68659, 157121, 3409, 10428, 10511, 59924, 68564, 1767, 27234, 222703, 44674, 6, 101486, 231824, 1789, 19203, 111307, 2538, 14710, 156045, 1242, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [52]:
print("encodinmmg start")
# valid_true_labels = valid_dataset["Score"]
# print(valid_dataset.num_rows)
valid_data={}
for i, lang in enumerate(lan):
    valid_data[lang]= valid_dataset[lang].map(add_encodings,load_from_cache_file=False, batch_size =10000)#load_from_cache_file=False,

encodinmmg start


Map:   0%|          | 0/550 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

Map:   0%|          | 0/78 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [53]:
for i, lang in enumerate(lan):
    valid_data[lang].set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])


In [54]:
test_data

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Split_Text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Split_Text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Split_Text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Split_Text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Split_Text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Split_Text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words', 'Split_Text', 'input_ids', 'attention_mask', 'labels'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['se

In [55]:
test_dataset

{'eng': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 250
 }),
 'esp': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 140
 }),
 'amh': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 95
 }),
 'arq': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 97
 }),
 'ary': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 71
 }),
 'hau': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 212
 }),
 'kin': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 102
 }),
 'mar': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 293
 }),
 'tel': Dataset({
     features: ['sentence_id', 'words', 'Split_Text'],
     num_rows: 130
 })}

In [56]:
for i, lang in enumerate(lan):
    test_data[lang].set_format(type='torch', columns=['input_ids', 'attention_mask','labels'])

In [57]:
# test_data = torch.utils.data.DataLoader( test_dataset, batch_size=batch_size )


In [58]:
torch.manual_seed(seed)
# initialize the model and provide the 'num_labels' used to create the classification layer
model = XLMRobertaForSequenceClassification.from_pretrained(roberta_version,num_labels=1)

print("model loded")

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model loded


In [59]:
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


cuda


In [60]:
torch.cuda.is_available()


True

In [61]:
torch.__version__

'2.0.1+cu117'

In [62]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [63]:

!nvidia-smi


Sun Jan 21 14:52:05 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     Off  | 00000000:81:00.0 Off |                  Off |
| 33%   28C    P8     5W / 260W |     28MiB / 24220MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [67]:
# set the model in 'train' mode and send it to the device
model.train().to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [68]:
model.config.num_labels

1

In [69]:
model.num_labels

1

In [70]:

optimizer = optim.AdamW([ { 'params':model.parameters(), 'lr':lr}])#,{'params':weight_layer, 'lr':l_lr},{'params':bias_layer, 'lr':l_lr}, {'params':crf_model.parameters(), 'lr':l_lr

# batch the train data so that each batch contains 4 examples (using 'batch_size')
train_dataloader={}
valid_dataloader={}
test_dataloader={}
for i, lang in enumerate(lan):
    # train_dataloader[lang] = torch.utils.data.DataLoader(train_data[lang], batch_size=batch_size)
    # valid_dataloader[lang] = torch.utils.data.DataLoader(valid_data[lang], batch_size=batch_size )
    test_dataloader[lang] = torch.utils.data.DataLoader(test_data[lang], batch_size=batch_size )


num_training_steps = (n_epochs * len(train_data))/16
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)

train_loss = []


In [72]:
lan

['eng', 'esp', 'amh', 'arq', 'ary', 'hau', 'kin', 'mar', 'tel']

In [73]:
# hyperparams_roberta_monolingual= {}
# for lang in lan:
#     hyperparams_roberta_monolingual[lan]=[


In [101]:
import mlflow
mlflow.set_tracking_uri("http://172.17.25.123:8080/")

In [111]:
# import mlflow
# mlflow.set_tracking_uri("http://172.17.25.123:8080/")
# model_checkpoint_path="/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/multiple_models_/XLM_roberta64_2e-05seed1/epoch_2"
model_checkpoint_path="xlm-roberta-large"
import tqdm
tqdmn = tqdm.notebook.tqdm
print("training start")
previous_total_valid_loss = sys.float_info.max
valid_min_loss = previous_total_valid_loss
early_stoping_flag=0
# p_directory = "/content/drive/MyDrive/textual_relatedness/models/xlm_64_1e_4seed11/"
hyperparameter_combinations=[
# hyperparameter_combinations = [
#     {'seed': 1, 'batch_size': 64, 'lr': 2e-4, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3, 'language_train_dataset':},
#     {'seed': 1, 'batch_size': 64, 'lr': 1e-4, 'n_epochs': 5,  'max_len': 64, 'early_stopping_patience': 3},
#     {'seed': 1, 'batch_size': 64, 'lr': 8e-5, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3},
#     {'seed': 1, 'batch_size': 64, 'lr': 1e-5, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3},
#     {'seed': 1, 'batch_size': 64, 'lr': 2e-5, 'n_epochs': 5,  'max_len': 64, 'early_stopping_patience': 3},
#     {'seed': 1, 'batch_size': 64, 'lr': 5e-5, 'n_epochs': 5, 'max_len': 64, 'early_stopping_patience': 3},
]
data_saved_path="2_stage_models_roberta_large_in_both_state"
for i, lang in enumerate(lan):
    # p_directory = f"2_stage_models_{lang}/XLM_roberta_64_2e-5_seed_1/"
    lang_directory = f"{lang}/XLM_roberta_64_2e-5_seed_1/"
    p_directory=os.path.join(data_saved_path, lang_directory)
    

# Start MLflow run
    # model = AutoModelForSequenceClassification.from_pretrained(roberta_version,num_labels=1)
    # model.train().to(device)
    with mlflow.start_run(run_name=f"1_stage_fine_tuning_Xlm_roberta_large_monolanguage {lang}") as run:
        # for param, value in hyperparams.items():
        #     mlflow.log_param(param, value)
        torch.manual_seed(seed)
        # model = XLMRobertaForSequenceClassification.from_pretrained(roberta_version,num_labels=1,ignore_mismatched_sizes=True)
        model = XLMRobertaForSequenceClassification.from_pretrained(model_checkpoint_path)
        model.train().to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
        num_training_steps = (n_epochs * len(train_data))/16
        lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=100, num_training_steps=num_training_steps)
        # lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)  # You can adjust the scheduler as needed
        # iterate through the data 'n_epochs' times
        train_accuracy_list = []
        valid_accuracy_list = []
        error=0
        for epoch in tqdmn(range(n_epochs)):
            count=0
            print("epoch", epoch)
            directory=p_directory+"epoch_"+str(epoch)
            print("\n")
            print(directory)
            #model = BertForTokenClassification.from_pretrained(directory)
            #model.train().to(device)
            current_loss = 0
            total_loss=0
            # iterate through each batch of t    he train data
            k=0
            model.train().to(device)          
            train_predictions=[]
            for i, batch in enumerate(tqdmn(train_dataloader[lang])):
                batch = { k: v.to(device) for k, v in batch.items()}
                # print(batch['input_ids'])
                count+=1
                outputs=model(**batch)
                # try:
                #     outputs = model(**batch)
                # except:
                #     error=error+1
                #     continue
                loss =outputs.loss
                logits=outputs.logits
                loss.backward()
                # predictions = torch.argmax(logits, dim=1)
                predictions=logits.cpu().detach()
                # print("predictions", predictions)
                train_predictions.extend(list(predictions))
                current_loss += loss.item()
                if i % 4 == 0 and i > 0:       # #  # #       #if i % 8 == 0 and i > 0:
                    # update the model using the optimizer
                    optimizer.step()
                    lr_scheduler.step()
                    # once we update the model we set the gradients to zero
                    optimizer.zero_grad()
                    # store the loss value for visualization
                    train_loss.append(current_loss/16)
                    total_loss=total_loss + current_loss/64"]
                    current_loss = 0
                # update the model one l2064469413403458ast time for this epoch
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # model.save_pretrained(directory)
            print(directory + ' files saved')
            print("error:", error)
            print('total_loss',total_loss)
            train_labels = train_pdf[lang]["Score"].values # Get your actual training labels
            # print(train_predictions)
            # print('**********')
            # print(train_labels)
            # train_predictions_clean = [item[0] for item in train_predictions]
            # print("train_predictions", train_predictions_clean)
            # train_final_predictions=[value.item() for value in train_predictions_clean]
            # print("train_final_predictions", train_final_predictions)
            print('*********')
            # print(train_final_predictions)
            # print(train_labels)
            # train_accuracy = round(pearsonr(train_final_predictions,train_labels)[0],2)
            # print("train_accuracy", train_accuracy)
            # train_accuracy_list.append(train_accuracy)
         # Save the model with hyperparameter details in the directory
            model_directory = os.path.join(p_directory, f"epoch_{epoch}")
            os.makedirs(model_directory, exist_ok=True)
            model.save_pretrained(model_directory)
            # mlflow.pytorch.save_model(model, model_directory)
            mlflow.log_artifact(local_path="Track A/"+lang+"/"+lang+"_train.csv", artifact_path="datasets")
            mlflow.log_metric("train_loss", total_loss)
            # mlflow.log_metric("train_accuracy", train_accuracy)
        #     validation####################################
            model.eval().to(device)
            valid_loss = []
            valid_current_loss = 0
            valid_total_loss=0
            # iterate through each batch of t    he train data
            valid_predictions=[]
            for i, batch in enumerate(tqdmn(valid_dataloader[lang])):
                #print(batch)
                batch = { k: v.to(device) for k, v in batch.items()}
                #print(batch["labels"].shape)
                #print(batch["input_ids"].shape)
                # send 'input_ids', 'attention_mask' and 'labels' to the model
                outputs = model(**batch)
# pandas==2.1.4
                # print(outputs)
                loss=outputs.loss
                logits = outputs.logits
                predictions=logits.cpu()
                # predictions = torch.argmax(logits, dim=1)
                valid_predictions.extend(predictions.detach())
                valid_current_loss += loss.item()
                #print(valid_current_loss)
                valid_loss.append(valid_current_loss/batch_size)
                valid_total_loss=valid_total_loss + valid_current_loss/batch_size
                valid_current_loss = 0
                    #print("valid_total_loss ",valid_total_loss)
            print("valid_total_loss ",valid_total_loss)
            valid_labels = dev_pdf[lang]["Score"].values  # Get your actual validation labels
            dev_predictions_clean = [item[0] for item in valid_predictions]
            dev_predictions_final=[value.item() for value in dev_predictions_clean]
            valid_accuracy =  round(pearsonr(dev_predictions_final,valid_labels)[0],2)
            print("valid_accuracy", valid_accuracy)
            valid_accuracy_list.append(valid_accuracy)
            mlflow.log_artifact(local_path="Track A/"+lang+"/"+lang+"_dev.csv", artifact_path="datasets")
            mlflow.log_metric("valid_total_loss ",valid_total_loss)
            mlflow.log_metric("valid_accuracy", valid_accuracy)
             # Log final accuracies for the last epoch
            # mlflow.log_metric("final_train_accuracy", train_accuracy_list[-1])
            mlflow.log_metric("final_valid_accuracy", valid_accuracy_list[-1])
            # valid_loss_epoch.append(valid_total_loss)
            if valid_total_loss < valid_min_loss:
                valid_min_loss = valid_total_loss
        #         model.save_pretrained(directory+"best_e_"+str(epoch))
            if previous_total_valid_loss < valid_total_loss:
                early_stoping_flag += 1
            else:
                early_stoping_flag=0
            previous_total_valid_loss = valid_total_loss
            if early_stoping_flag == early_stoping_patience:
                print("early_stoping occure")
                break
print('All files saved')

client = MlflowClient()
best_run = client.search_runs(order_by=["metrics.valid_accuracy desc"]).iloc[0]
best_model_uri = best_run.info.artifact_uri + "/models"
print(f"The best model is stored at: {best_model_uri}")

# Command to see all the metrics on MLflow UI
print(f"Visit MLflow UI: {mlflow.get_tracking_uri()}")

training start


  0%|          | 0/5 [00:00<?, ?it/s]

epoch 0


2_stage_models/eng/XLM_roberta_64_2e-5_seed_1/epoch_0


  0%|          | 0/310 [00:00<?, ?it/s]

2_stage_models/eng/XLM_roberta_64_2e-5_seed_1/epoch_0 files saved
error: 0
total_loss 0.10724538621434476
*********


...

  0%|          | 0/66 [00:00<?, ?it/s]

2_stage_models/tel/XLM_roberta_64_2e-5_seed_1/epoch_4 files saved
error: 0
total_loss 0.017465975193772465
*********


  0%|          | 0/8 [00:00<?, ?it/s]

valid_total_loss  0.004783414566190913
valid_accuracy 0.82
All files saved


NameError: name 'MlflowClient' is not defined

In [116]:
len(test_dataloader['amh'])

6

In [154]:
# print("training start")
# previous_total_valid_loss = sys.float_info.max
# valid_min_loss = previous_total_valid_loss
# early_stoping_flag=0

# p_directory = "/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/"

# # iterate through the data 'n_epochs' times
# for epoch in range(n_epochs):
#     print(epoch)
#     directory=p_directory+"epoch_"+str(epoch)
#     print(directory)

#     #model = BertForTokenClassification.from_pretrained(directory)
#     #model.train().to(device)

#     current_loss = 0
#     total_loss=0
#     # iterate through each batch of t    he train data
#     k=0
#     for i, batch in enumerate(train_data):

#         # move the batch tensors to the same device as the
#         #print(batch)
#         batch = { k: v.to(device) for k, v in batch.items() }
#         # print(batch)


#         outputs = model(**batch)
#         # print(outputs)
#         # print(outputs.loss)
#         # print(outputs.logits.shape)
#         # the outputs are of shape (loss, logits)
#         loss =outputs.loss

#         loss.backward()
#         # NOTE: if we append `loss` (a tensor) we will force the GPU to save
#         # the loss into its memory, potentially filling it up. To avoid this
#         # we rather store its float value, which can be accessed through the
#         # `.item` method
#         current_loss += loss.item()

#         if i % 4 == 0 and i > 0:       # #  # #       #if i % 8 == 0 and i > 0:
#             # update the model using the optimizer
#             optimizer.step()
#             lr_scheduler.step()
#             # once we update the model we set the gradients to zero
#             optimizer.zero_grad()
#             # store the loss value for visualization
#             train_loss.append(current_loss/16)
#             total_loss=total_loss + current_loss/64
#             current_loss = 0


#         # update the model one last time for this epoch
#     optimizer.step()
#     lr_scheduler.step()
#     optimizer.zero_grad()
#     model.save_pretrained(directory)columns
#     print(directory + ' files saved')
#     print('total_loss',total_loss)


# #     validation###########################################################

#     model.eval().to(device)
#     valid_loss = []
#     valid_current_loss = 0
#     valid_total_loss=0

#     # iterate through each batch of t    he train data
#     for i, batch in enumerate(tqdm(valid_data)):

#         #print(batch)
#         batch = { k: v.to(device) for k, v in batch.items() }

#         #print(batch["labels"].shape)
#         #print(batch["input_ids"].shape)
#         # send 'input_ids', 'attention_mask' and 'labels' to the model
#         outputs = model(**batch)
#         # print(outputs)
#         loss=outputs.loss


#         valid_current_loss += loss.item()
#         #print(valid_current_loss)



#         valid_loss.append(valid_current_loss/batch_size)
#         valid_total_loss=valid_total_loss + valid_current_loss/batch_size
#         valid_current_loss = 0
#             #print("valid_total_loss ",valid_total_loss)

#     print("valid_total_loss ",valid_total_loss)
#     # valid_loss_epoch.append(valid_total_loss)
#     if valid_total_loss < valid_min_loss:
#         valid_min_loss = valid_total_loss
# #         model.save_pretrained(directory+"best_e_"+str(epoch))


#     if previous_total_valid_loss < valid_total_loss:
#         early_stoping_flag += 1
#     else:
#         early_stoping_flag=0
#     previous_total_valid_loss = valid_total_loss


#     if early_stoping_flag == early_stoping_patience:
#         print("early_stoping occure")
#         break



# #  target_names = ['O', 'B-PERSON', 'I-PERSON', 'B-LOCATION', 'I-LOCATION', 'B-ORGANIZATION', 'I-ORGANIZATION', 'B-FESTIVAL', 'I-FESTIVAL', 'B-GAME', 'I-GAME', 'B-LANGUAGE', 'I-LANGUAGE', 'B-LITERATURE', 'I-LITERATURE', 'B-MISC', 'I-MISC', 'B-NUMEX', 'I-NUMEX', 'B-RELIGION', 'I-RELIGION', 'B-TIMEX', 'I-TIMEX']

# # save vocabulary of the tokenizer
# #tokenizer.save_vocabulary(directory)
# # save the model weights and its configuration file

# print('All files saved')



training start
0
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_0
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_0 files saved
total_loss 0.765174181186012


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.60it/s]


valid_total_loss  0.020648444653488696
1
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_1
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_1 files saved
total_loss 0.24859311032196274


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.56it/s]


valid_total_loss  0.016335177060682327
2
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_2
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_2 files saved
total_loss 0.1621586470209877


...

valid_total_loss  0.021607325441436842
13
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_13
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_13 files saved
total_loss 0.09641751476920035


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.55it/s]


valid_total_loss  0.021607325441436842
14
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_14
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/Track A/models/XLM_2e5_16_seed7/epoch_14 files saved
total_loss 0.09641751476920035


100%|███████████████████████████████████████████| 24/24 [00:02<00:00, 10.53it/s]

valid_total_loss  0.021607325441436842
All files saved


In [ ]:
if not os.path.exists(zip_directory):
        os.makedirs(zip_directory)
# best_model_path="/content/drive/MyDrive/textual_relatedness/models/epoch_3"#81.70
best_model_path="/content/drive/MyDrive/textual_relatedness/models/epoch_5"


num_labels = 1
model_max_length = max_len
batch_size= 16
min_batch_size=4
batch_size_c = batch_size / min_batch_size

In [113]:

# # Prediction
# def  prediction(best_model_path, eval_data):


#     predicted=[]
#     tokanised_exam=columns
#     true_labels=[]

#     confusion = torch.zeros(num_labels, num_labels)
#     predicted_for_classification_report=[]

#     ##################################################
#     #load best model

#     # for i, lang in enumerate(test_lan)
#     print(best_model_path)

#     model = RobertaForSequenceClassification.from_pretrained(best_model_path)



#     model = model.eval().to(device)
#     print("model device")
#     # batch the train data so that each batch contains 4 examples (using 'batch_size')
#     #test_data = torch.utils.data.DataLoader(test_dataset, batch_size=16)





#     predicted=[]
#     true_labels=[]


#     predicted_for_classification_report=[]
#     # iterate through each batch of t    he train data
#     for i, batch in enumerate(tqdmn(test_dataloader)):
    

#         with torch.no_grad():
# #             print(batch)
#             batch = { k: v.to(device) for k, v in batch.items() }
#             # batch_labels = batch.pop("labels")
#             outputs = model(**batch)

#             logits = outputs.logits
#             #print(logits.shape)



#         # get the predicted values
#         pred_values =logits.tolist()
#         #print(pred_values)
#         # true_labels += tags[idx][1:length-1].tolist()
#         # true_labels_list.append(tags[idx][:length].tolist())

#         #pred_values = pred_values.tolist()

#         # print(pred_values)
#         flattened_pred_values = [item for sublist in pred_values for item in sublist]

#         print(flattened_pred_values)
#         predicted +=flattened_pred_values

#     # print(Counter(predicted))

#     return predicted #here predicred is flat


In [176]:
for lang in test_lan:
    print(best_models[lang])

/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/amh/XLM_roberta_64_2e-5_seed_1/epoch_0
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/arq/XLM_roberta_64_2e-5_seed_1/epoch_4
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/ary/XLM_roberta_64_2e-5_seed_1/epoch_0
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/eng/XLM_roberta_64_2e-5_seed_1/epoch_1
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/esp/XLM_roberta_64_2e-5_seed_1/epoch_2
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/hau/XLM_roberta_64_2e-5_seed_1/epoch_1
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/kin/XLM_roberta_64_2e-5_seed_1/epoch_2
/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval202

In [115]:
test_pdf['amh']

,PairID,Text,Split_Text
0,Pair_ID_amh_dev_1,ታህሳስ 5 ፣2013/ በኢትዮጵያ ተጨማሪ 473 ሰዎች በኮሮና ቫይረስ ተያ...,[ታህሳስ 5 ፣2013/ በኢትዮጵያ ተጨማሪ 473 ሰዎች በኮሮና ቫይረስ ተ...
1,Pair_ID_amh_dev_2,በፖለቲካ እምነት መሆን የለበትም ፡፡\tመልሱ መሆን የለበትም ነው ፡፡,[በፖለቲካ እምነት መሆን የለበትም ፡፡\tመልሱ መሆን የለበትም ነው ፡፡]
2,Pair_ID_amh_dev_3,መልካም በአል ለመላው ክርስቶስትና እምነት ተከታዮች በሙሉ ?\tለመላው የ...,[መልካም በአል ለመላው ክርስቶስትና እምነት ተከታዮች በሙሉ ?\tለመላው ...
3,Pair_ID_amh_dev_4,የኢትዮጵያ ገበሬዎች እርዳታ እንደሚያስፈልጋቸው ገልፀዋል ።\tየኢትዮጵያ ...,[የኢትዮጵያ ገበሬዎች እርዳታ እንደሚያስፈልጋቸው ገልፀዋል ።\tየኢትዮጵያ...
4,Pair_ID_amh_dev_5,ዱላ ይዞ ነበር እንዴ ?\tግን የዛሬው የወያኔ ዱላ እንዴት ነበር ?,[ዱላ ይዞ ነበር እንዴ ?\tግን የዛሬው የወያኔ ዱላ እንዴት ነበር ?]
...,...,...,...
90,Pair_ID_amh_dev_91,ይሖዋ “የእውነት አምላክ ” ነው ።\tእንዲህ ማድረግ የሚችለው ደግሞ ይሖ...,[ይሖዋ “የእውነት አምላክ ” ነው ።\tእንዲህ ማድረግ የሚችለው ደግሞ ይ...
91,Pair_ID_amh_dev_92,ክረምትና በጋ በምስራቅ አፍሪካ ።\tበብዛት በምስራቅ አፍሪካ ይገኛሉ ።,[ክረምትና በጋ በምስራቅ አፍሪካ ።\tበብዛት በምስራቅ አፍሪካ ይገኛሉ ።]
92,Pair_ID_amh_dev_93,ይህን ያህል እርግጠኛ እንዲሆን ያደረገው ምንድን ነው ?\tደስተኛ እንዲሆ...,[ይህን ያህል እርግጠኛ እንዲሆን ያደረገው ምንድን ነው ?\tደስተኛ እንዲ...
93,Pair_ID_amh_dev_94,ያ የበላ ሰው ከሕዝቡ መካከል ተለይቶ ይጠፋልና ።\tመንግስት ግን ፣ “ለ...,[ያ የበላ ሰው ከሕዝቡ መካከል ተለይቶ ይጠፋልና ።\tመንግስት ግን ፣ “...


In [131]:

# Prediction
def  prediction(best_model_path, eval_data, test_pdf, results_path, zip_file_path):
    # predicted=[]
    # tokanised_exam=columns
    # true_labels=[]
    confusion = torch.zeros(num_labels, num_labels)
    predicted_for_classification_report=[]
    ##################################################
    #load best model
    # for lang, best_models in zip(test_lan,best_model_path):
    final_results_path={}
    zip_files_path={}
    for lang in test_lan:
        print(f"path of the {lang} model is: {best_model_path}")
        # print
        model = RobertaForSequenceClassification.from_pretrained(best_models[lan])  
        model = model.eval().to(device)
        print("model device")
        # batch the train data so that each batch contains 4 examples (using 'batch_size')
        #test_data = torch.utils.data.DataLoader(test_dataset, batch_size=16)
        predicted=[]
        predictions={}
        true_labels=[]
        predicted_for_classification_report=[]
        # iterate through each batch of t    he train data
        for i, batch in enumerate(tqdmn(test_dataloader[lang])):
            # csv_file_path= f"pred_{lang}_a.csv"
            # final_results_path[lang]=os.path.join(results_path, csv_file_path)

            with torch.no_grad():
    #             print(batch)
                batch = { k: v.to(device) for k, v in batch.items() }
                # batch_labels = batch.pop("labels")
                outputs = model(**batch)
                logits = outputs.logits
                #print(logits.shape)
                # get the predicted values
            pred_values =logits.tolist()
            #print(pred_values)
            # true_labels += tags[idx][1:length-1].tolist()
            # true_labels_list.append(tags[idx][:length].tolist())
                #pred_values = pred_values.tolist()
                # print(pred_values)
            flattened_pred_values = [item for sublist in pred_values for item in sublist]
            # print(flattened_pred_values)
            predicted +=flattened_pred_values
            
        print(len(predicted))   
        print(test_pdf[lang])
        test_pdf[lang]["Pred_Score"] = predicted
        csv_file_path= f"pred_{lang}_a.csv"
        final_results_path[lang]= os.path.join(results_path,csv_file_path) 
        test_pdf[lang][['PairID','Pred_Score']].to_csv(final_results_path[lang],index=False)
    #     if not os.path.exists(zip_file_path):
    #         os.makedirs(zip_file_path)
    #     zip_files_path[lang]=os.path.join(zip_file_path, f"pred_{lang}_a.zip")
    #     with zipfile.ZipFile( zip_files_path[lang], 'w') as zipf:
    # # Add the CSV file to the zip file with a new name (you can change it if needed)
    #         zipf.write(zip_files_path[lang], arcname=os.path.basename(final_results_path[lang]))

    # print(f"CSV file '{csv_file_path}' successfully zipped to '{zip_file_path}'.")

            
            # return predicted #here predicred is flat

In [135]:
for lang in test_lan:
    compressed_file_path[lang]=f"Results/2_stage_results_XLM_roberta/zip_files/pred_{lang}_a.zip"
    file_path[lang]=f"Results/2_stage_results_XLM_roberta/csv_files/pred_{lang}_a.csv"
    !zip compressed_file_path[lang] file_path[lang]

NameError: name 'compressed_file_path' is not defined

In [132]:
prediction(best_model_path, test_dataloader, test_pdf, "Results/2_stage_results_XLM_roberta/csv_files/", "Results/2_stage_results_XLM_roberta/zip_files/")

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


path of the amh model is: ['/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/amh/XLM_roberta_64_2e-5_seed_1/epoch_0', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/arq/XLM_roberta_64_2e-5_seed_1/epoch_4', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/ary/XLM_roberta_64_2e-5_seed_1/epoch_0', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/eng/XLM_roberta_64_2e-5_seed_1/epoch_1', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/esp/XLM_roberta_64_2e-5_seed_1/epoch_2', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/hau/XLM_roberta_64_2e-5_seed_1/epoch_1', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/kin/XLM_roberta_64_2e-5_seed_1/epoch_2', '/home/naive123/nlp/Sumit/Te

  0%|          | 0/6 [00:00<?, ?it/s]

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


95
                PairID                                               Text  \
0    Pair_ID_amh_dev_1  ታህሳስ 5 ፣2013/ በኢትዮጵያ ተጨማሪ 473 ሰዎች በኮሮና ቫይረስ ተያ...   
1    Pair_ID_amh_dev_2       በፖለቲካ እምነት መሆን የለበትም ፡፡\tመልሱ መሆን የለበትም ነው ፡፡   
2    Pair_ID_amh_dev_3  መልካም በአል ለመላው ክርስቶስትና እምነት ተከታዮች በሙሉ ?\tለመላው የ...   
3    Pair_ID_amh_dev_4  የኢትዮጵያ ገበሬዎች እርዳታ እንደሚያስፈልጋቸው ገልፀዋል ።\tየኢትዮጵያ ...   
4    Pair_ID_amh_dev_5        ዱላ ይዞ ነበር እንዴ ?\tግን የዛሬው የወያኔ ዱላ እንዴት ነበር ?   
..                 ...                                                ...   
90  Pair_ID_amh_dev_91  ይሖዋ “የእውነት አምላክ ” ነው ።\tእንዲህ ማድረግ የሚችለው ደግሞ ይሖ...   
91  Pair_ID_amh_dev_92      ክረምትና በጋ በምስራቅ አፍሪካ ።\tበብዛት በምስራቅ አፍሪካ ይገኛሉ ።   
92  Pair_ID_amh_dev_93  ይህን ያህል እርግጠኛ እንዲሆን ያደረገው ምንድን ነው ?\tደስተኛ እንዲሆ...   
93  Pair_ID_amh_dev_94  ያ የበላ ሰው ከሕዝቡ መካከል ተለይቶ ይጠፋልና ።\tመንግስት ግን ፣ “ለ...   
94  Pair_ID_amh_dev_95  በሌላ በኩል የላቀ የሻምፒዮናነነት ፉክክር እንደሚደረግበት የሚጠበቀው 20...   

                                           Split_Text  Pred_Score  
0   

...

  0%|          | 0/19 [00:00<?, ?it/s]

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


293
            PairID                                               Text  \
0    MAR-dev-00001  "या प्रसंगी जाहीर सभेला संबोधित करताना ते म्हण...   
1    MAR-dev-00002  "गौतम गंभीरची संपती 147 कोटी रुपये असल्याची मा...   
2    MAR-dev-00003  "खुद्द शरद पवार यांनी याबाबत स्थानिक नेत्यांचा...   
3    MAR-dev-00004  "पेट्रोकेमिकल प्रकल्प आणून येथील फळबागायती आम्...   
4    MAR-dev-00005  "त्यामुळे दक्षिण आफ्रिकेकडून सर्वाधिक बळी मिळव...   
..             ...                                                ...   
288  MAR-dev-00289  "पंतप्रधानांनी “कथन” म्हणून जे वर्णन केले ते भ...   
289  MAR-dev-00290  "त्यावर आता राज्याच्या विधान परिषदेचे विरोधी प...   
290  MAR-dev-00291  "या योजनेचे उद्दीष्ट सर्व घरांना शक्ती प्रदान ...   
291  MAR-dev-00292  "शिवसेना व भाजपमध्ये युती झाल्यानंतर दोन्ही पक...   
292  MAR-dev-00293  "त्यांनी स्वच्छ भारत मिशन, बेटी बाचाओ - बेटी प...   

                                            Split_Text  Pred_Score  
0    ["या प्रसंगी जाहीर सभेला संबोधित करताना ते म्

  0%|          | 0/9 [00:00<?, ?it/s]

130
            PairID                                               Text  \
0    TEL-dev-00001  "బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా నేడు రెండోరో...   
1    TEL-dev-00002  "ఈ ఏడాది జనవరి 1వ తేదీ నాటికి ఢిల్లీలో 10,76,4...   
2    TEL-dev-00003  "ప్రతిపక్షాలు అనవసర రాద్ధాంతం చేస్తున్నాయి : హ...   
3    TEL-dev-00004  "హైదరాబాద్ : కాంగ్రెస్ పార్టీ అధికారంలోకి రాగా...   
4    TEL-dev-00005  "ప్రజారాజధానిగా రూపుదిద్దేందుకు అవసరమైన పెట్టు...   
..             ...                                                ...   
125  TEL-dev-00126  "రేలంగి నరసింహారావు వంటి దర్శకులు సినిమాలు చేయ...   
126  TEL-dev-00127  "సూర్య డ్యూయల్ రోల్ చేసిన ఈ సినిమా ఆశించిన స్థ...   
127  TEL-dev-00128  "ఇండియన్ ప్రీమియర్ లీగ్ – 2018 విజయవంతంగా కొనస...   
128  TEL-dev-00129  "వలస కార్మికుల రైళ్లను బెంగాల్ అనుమతించడంలేదు:...   
129  TEL-dev-00130  "తెలంగాణలోకి మావోలు చొరబడే అవకాశం లేదు: డీజీపీ...   

                                            Split_Text  Pred_Score  
0    ["బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా నేడు రెండో

In [ ]:
valid_predicted = prediction(best_model_path, valid_data)

/content/drive/MyDrive/textual_relatedness/models/epoch_4
model device


  0%|          | 0/69 [00:00<?, ?it/s]

[0.8677147030830383, 0.9685263633728027, 0.9671577215194702, 0.8463200330734253, 0.7330904006958008, 0.9127029180526733, 0.8296362161636353, 0.8470741510391235, 0.8778345584869385, 0.8357004523277283, 0.8777657747268677, 0.8939412236213684, 0.9307860732078552, 0.7076771855354309, 0.9769800305366516, 0.7721829414367676]
[0.8665282726287842, 0.6413089632987976, 0.8545132875442505, 0.8522595763206482, 0.8274969458580017, 0.715861976146698, 0.8624814748764038, 0.6947680711746216, 0.8666069507598877, 0.8458764553070068, 0.7459472417831421, 0.7477755546569824, 0.7157862782478333, 0.9091516733169556, 0.8417872786521912, 0.7588858008384705]
[0.8460123538970947, 0.9143412709236145, 0.7486365437507629, 0.8199021816253662, 0.9296796321868896, 0.9182645082473755, 0.8750166296958923, 0.736318826675415, 0.8660807609558105, 0.791685938835144, 0.8380727767944336, 0.5579389929771423, 0.7509849667549133, 0.8312090039253235, 0.6925581693649292, 0.8871711492538452]
[0.9125977158546448, 0.7389199733734131,

In [93]:

best_model_path=["/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/amh/XLM_roberta_64_2e-5_seed_1/epoch_0",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/arq/XLM_roberta_64_2e-5_seed_1/epoch_4",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/ary/XLM_roberta_64_2e-5_seed_1/epoch_0",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/eng/XLM_roberta_64_2e-5_seed_1/epoch_1",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/esp/XLM_roberta_64_2e-5_seed_1/epoch_2",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/hau/XLM_roberta_64_2e-5_seed_1/epoch_1",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/kin/XLM_roberta_64_2e-5_seed_1/epoch_2",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/mar/XLM_roberta_64_2e-5_seed_1/epoch_0",
		"/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/tel/XLM_roberta_64_2e-5_seed_1/epoch_3"
                ]
		
		
		

In [96]:
test_lan=['amh', 'arq', 'ary', 'eng', 'esp', 'hau', 'kin', 'mar', 'tel']
best_models={}
for lan, best_model in zip(test_lan, best_model_path):
    best_models[lan]=best_model

In [97]:
best_models

{'amh': '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/amh/XLM_roberta_64_2e-5_seed_1/epoch_0',
 'arq': '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/arq/XLM_roberta_64_2e-5_seed_1/epoch_4',
 'ary': '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/ary/XLM_roberta_64_2e-5_seed_1/epoch_0',
 'eng': '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/eng/XLM_roberta_64_2e-5_seed_1/epoch_1',
 'esp': '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/esp/XLM_roberta_64_2e-5_seed_1/epoch_2',
 'hau': '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/hau/XLM_roberta_64_2e-5_seed_1/epoch_1',
 'kin': '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/kin/XLM_roberta_64_2e-5_seed_1/epoch_2',

In [87]:
prediction(best_model_path, test_dataloader, test_pdf, "Results/2_stage_results_XLM_roberta", "Results/2_stage_results_XLM_roberta/Zip_files")

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


path of the amh model is: ['/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/amh/XLM_roberta_64_2e-5_seed_1/epoch_0', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/arq/XLM_roberta_64_2e-5_seed_1/epoch_4', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/ary/XLM_roberta_64_2e-5_seed_1/epoch_0', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/eng/XLM_roberta_64_2e-5_seed_1/epoch_1', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/esp/XLM_roberta_64_2e-5_seed_1/epoch_2', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/hau/XLM_roberta_64_2e-5_seed_1/epoch_1', '/home/naive123/nlp/Sumit/Textural_Relateness/Semantic_Relatedness_SemEval2024/2_stage_models/kin/XLM_roberta_64_2e-5_seed_1/epoch_2', '/home/naive123/nlp/Sumit/Te

  0%|          | 0/9 [00:00<?, ?it/s]

[0.6853533983230591, 0.33357831835746765, 0.6082204580307007, 0.5725260376930237, 0.3369078040122986, 0.672784686088562, 0.3820137679576874, 0.5785830020904541, 0.46819403767585754, 0.6229170560836792, 0.6752933263778687, 0.6368311643600464, 0.6820259094238281, 0.6430413722991943, 0.4802967607975006, 0.26543769240379333]
[0.611850380897522, 0.4251039922237396, 0.4825853407382965, 0.40129056572914124, 0.6453731060028076, 0.48656055331230164, 0.4750514626502991, 0.508009672164917, 0.6785788536071777, 0.4598662555217743, 0.4739186465740204, 0.40198585391044617, 0.6958822011947632, 0.6458245515823364, 0.33412542939186096, 0.6226108074188232]
[0.6545341610908508, 0.6752777099609375, 0.31604403257369995, 0.6305542588233948, 0.5135071873664856, 0.7008060812950134, 0.5528655648231506, 0.6815924048423767, 0.3599611818790436, 0.5180437564849854, 0.6797119379043579, 0.6715090870857239, 0.3374292850494385, 0.27171221375465393, 0.34256649017333984, 0.6294949054718018]
[0.3774915933609009, 0.5471764

IsADirectoryError: [Errno 21] Is a directory: 'Results/2_stage_results_XLM_roberta/Zip_files'

In [ ]:
test_predicted={}
for lang in test_lan:
    valid_predicted = prediction(best_models, valid_data)

In [ ]:
valid_predicted[0:10]

[0.8677147030830383,
 0.9685263633728027,
 0.9671577215194702,
 0.8463200330734253,
 0.7330904006958008,
 0.9127029180526733,
 0.8296362161636353,
 0.8470741510391235,
 0.8778345584869385,
 0.8357004523277283]

In [ ]:
test_predicted[:10]
print(len(test_predicted))
test_predicted
test_pdf["Pred_Score"] = test_predicted

250


In [182]:
test_pdf

{'eng':            PairID                                               Text  \
 0    ENG-dev-0000  The story is gripping and interesting.\nIt's a...   
 1    ENG-dev-0001  The majority of Southeast Alaska 's area is pa...   
 2    ENG-dev-0002  and from your post i think you are to young to...   
 3    ENG-dev-0003  The film 's success also made Dreamworks Anima...   
 4    ENG-dev-0004  I am still confused about how I feel about thi...   
 ..            ...                                                ...   
 245  ENG-dev-0245  thats just how they are :( its a shame lol\nIt...   
 246  ENG-dev-0246  I feel sorry for the books that I will read af...   
 247  ENG-dev-0247  Uwe Seeler -LRB- born 5 November 1936 in Hambu...   
 248  ENG-dev-0248  Waco is a city in and the county seat of McLen...   
 249  ENG-dev-0249  religious extremism continues in pakistan desp...   
 
                                             Split_Text  
 0    [The story is gripping and interesting., It's ...  

In [ ]:
test_pdf[['PairID','Pred_Score']].to_csv("pred_eng_a.csv",index=False)

In [ ]:
# For testing
#label post allignment
def generate_score(predicted,true_labels):
  print("Pearson Correlation:", round(pearsonr(true_scores,pred_scores)[0]))

In [ ]:
generate_score( valid_predicted,valid_true_labels)

Pearson Correlation: 1


In [ ]:
for i,j in zip(valid_true_labels,valid_predicted):
  print(i,j)

1.0 0.8677147030830383
1.0 0.9685263633728027
1.0 0.9671577215194702
0.97 0.8463200330734253
0.97 0.7330904006958008
0.97 0.9127029180526733
0.94 0.8296362161636353
0.94 0.8470741510391235
0.94 0.8778345584869385
0.94 0.8357004523277283
0.94 0.8777657747268677
0.94 0.8939412236213684
0.94 0.9307860732078552
0.94 0.7076771855354309
0.94 0.9769800305366516
0.94 0.7721829414367676
0.94 0.8665282726287842
0.93 0.6413089632987976
0.91 0.8545132875442505
0.91 0.8522595763206482
0.91 0.8274969458580017
0.91 0.715861976146698
0.91 0.8624814748764038
0.91 0.6947680711746216
0.91 0.8666069507598877
0.91 0.8458764553070068
0.91 0.7459472417831421
0.91 0.7477755546569824
0.91 0.7157862782478333
0.91 0.9091516733169556
0.91 0.8417872786521912
0.91 0.7588858008384705
0.91 0.8460123538970947
0.91 0.9143412709236145
0.91 0.7486365437507629
0.91 0.8199021816253662
0.91 0.9296796321868896
0.91 0.9182645082473755
0.91 0.8750166296958923
0.91 0.736318826675415
0.91 0.8660807609558105
0.91 0.79168593883514